In [1]:
import pandas as pd
import numpy as np
import base64
from functools import reduce

In [4]:
test_data = pd.read_csv("test_data_features.csv") #saved erlier

In [13]:
train_data = pd.read_csv("train_data_features.csv")

In [23]:
%%time
train = pd.read_csv("kaggle_train_data_tab_new.csv", sep='\t')

CPU times: user 2.94 s, sys: 322 ms, total: 3.26 s
Wall time: 9.13 s


In [55]:
train_data['Prediction'] = train['Prediction']

In [976]:
test_data['encoded_len'] = test['PageBase64'].apply(len)

In [25]:
train_data['PageBase64'] = train_data['PageBase64'].apply(lambda x: base64.b64decode(x))

In [6]:
test_data['PageBase64'] = test_data['PageBase64'].apply(lambda x: base64.b64decode(x))

In [972]:
train_data

,Id,Prediction,Url,PageBase64,br_number,href_number,img_number,span_number,table_number,html_len,...,a_text_len,a_text,title_text,title_len,bad_words_num,tr_number,encoded_len,compression_coef,title>12,strog_number
0,-9222401963271173253,0,http://lawleader.ru/docs/32/,b'\x0c\xe0\x93`\xf1!\xb6iOP\x12\xc8\x0b\xff\xd...,167,59,8,5,1,62268,...,1459,главная о компании услуги бизнес-договоры ново...,"договор займа, договоры, договора",34,1,1,13622,0.218764,0,24
1,-9221399504663603656,0,http://alterhost.ruwww.alterhost.ruwww.user.kz...,"b""\x0c\xe0\x93`\xf1!\xb6iOP\x12\xc8\x0b\xff\xd...",1,80,60,46,0,39936,...,1337,главная форумы и обсуждения блоги творчество о...,uzru – миры общения,21,2,0,10187,0.255083,0,0
2,-9219733223042265364,0,http://www.ferio.ru/c_1-acura-legend.html,b'\x0c\xe0\x93`\xf1\x07L\xc2\xcfP\x12\xc8\x0b\...,69,138,5,1,38,44465,...,2219,\nсанкт-петербург \nсибирский фо \nюжный фо \n...,ferio.ru: ремонт и автозапчасти acura legend,46,4,149,8743,0.196627,0,0
3,-9218561774155397294,0,http://fshops.otzyv.ru/read.php?id=277,"b""\x0c\xe0\x93`\xf1\x07L\xc2\xcfP\x12\xc8\x0b\...",68,72,7,7,57,29931,...,1016,форумы туризм кино авто магазины контакты фор...,как повесить штору?,20,1,108,7013,0.234306,0,0
4,-9216163564171210203,1,http://penhea895.narod.ru/familiya-shushmarche...,b'\x0c\xe0\x93`\xf1!\xb6iOP\x12\xc8\x0b\xff\xd...,4,79,7,33,3,16707,...,1246,рамблер поиск ещё авторамблер аудио афиша блог...,фамилия шушмарченко - французские фамилии,42,0,3,4725,0.282816,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7039,9214475703562853998,1,http://glenadhy.tk/intimnie-znakomstva-v-smole...,b'\x86\xd9\xa5\x85\xe6\x9d\xb6+ex\xb8\x9fzg\x9...,34,47,14,1,7,20917,...,956,сайты знакомств в донецке знакомства г алмат...,lifemeet социальная сеть секс знакомств - реа...,76,40,15,6527,0.312043,0,1
7040,9214944781406040665,1,http://eicoepru.co.cc/probit-vin-pontiac/rossh...,b'\x86\xd9\xa5\xc6ig\xb2\x1bm\xa7\xff\xf0\xc3\...,2,15,4,0,3,15261,...,425,здесь пробить по vin производство авто vin код...,росшифровка vin daim — vin автомобилей утоплен...,52,0,10,5456,0.357513,0,2
7041,9217854286139025791,1,http://myflyuux.co.cc/96/46.html,b'\x86\xd9\xa5\x85\xe6\x9d\xb6+e{\xfbb\xb6W\xa...,5,20,3,1,1,15361,...,628,\n телефонный справочник киева по фамилии поис...,ялта.телефонный справочник | детективное агенс...,50,0,5,5304,0.345290,0,0
7042,9220612737580945795,1,http://nirpeaqo.20x.cc/?page=59,"b""\x86\xd9\xa5\x85\xe6\x9d\xb6+ezx\xab\xa5\xe6...",41,169,0,0,12,28990,...,6648,http://zzsklad.nm.ru/#639 скачать драйвер acor...,гостевуха сайта nirpeaqo.20x.cc - страница 59,46,93,50,8325,0.287168,0,0


In [8]:
train_data[['PageBase64', 'Url']]['Url'][7040]

'http://eicoepru.co.cc/probit-vin-pontiac/rosshifrovka-vin-daim-381/'

In [36]:
from bs4 import BeautifulSoup
import re

In [103]:
def get_tag(html, tag):
    
    soup = BeautifulSoup(html, 'html')
    text = str()
    
    for searched_tag in soup.find_all(tag):
        text += searched_tag.text
        
    return text

In [11]:
def count_tag(html, tag):
#     global a
#     a += 1
#     if (a % 500 == 0):
#         print(a)
    soup = BeautifulSoup(html, 'html')
    count = 0
    
    for searched_tag in soup.find_all(tag):
        count += 1
        
    return count

In [ ]:
train_data['a_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'a'))

In [87]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score, GroupKFold
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

In [88]:
def flatten(x):
    
    result = []
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str):
            result.extend(flatten(elem))
        else:
            result.append(elem)
            
    return result

In [89]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [90]:
def combinations(params):
    
    list_ = []
    for value in params.values():
        list_.append(value)

    tmp = reduce(lambda x, y: list_concat(x, y), list_)

    res = []
    for elem in tmp:
        param_list = flatten(elem)
        param_dict = dict(zip(params.keys(), param_list))
        res.append(param_dict)
        
    return res

In [91]:
def validation(X_train, train_target, model, params, folds_gen_func, folds_num=5, thresholds=[0.32], **kwargs):
    
    scaler = StandardScaler()
    scaler.fit(X_train) 
    
    main_res = []
    for param_set in combinations(params):
        
        print(param_set)        
        exact_model = model(**param_set) 
        
        fold_generator = folds_gen_func(folds_num)
        
        for th in thresholds:
#           th = 0.27
            print('th = ', th)
            res = []
            for train_index, test_index in fold_generator.split(X_train, train_target, **kwargs):

                exact_model.fit(scaler.transform(X_train[train_index]), train_target[train_index])

                y_pred = [0 if val < th else 1 for val in exact_model.predict_proba(scaler.transform(X_train[test_index]))[:,1]]

                score = metrics.f1_score(train_target[test_index], y_pred, average='weighted')
    #               print('threshold = {}, score = {}'.format(th, score))

                res.append(score)
        #                                       exact_model.predict(scaler.transform(X_train[test_index]))))


            mean = sum(res)/len(res)
            print(mean)
#             print('threshold = {}, score = {}'.format(th, mean))
            main_res.append((mean, param_set, th))
    
    best = main_res[np.argmax([res[0] for res in main_res])]
    print('--------max-------')
    print(best)
    
    return best

In [92]:
def predict(X_train, X_test, train_target, model, scaler=None, th=0.30, **kwargs):
    
    curr_model = model(**kwargs)
    
    if scaler is not None:
        
        your_scaler = scaler()
        your_scaler.fit(X_train)
        X_train = your_scaler.transform(X_train)
        X_test = your_scaler.transform(X_test)
        
    curr_model.fit(X_train, train_target)
#     y_pred = curr_model.predict(X_test)
#     th = 0.35
    print('Threshold = ', th )#,'Params = ', *kwargs.values)
    y_pred = [0 if val < th else 1 for val in curr_model.predict_proba(X_test)[:,1]]
    return y_pred

In [93]:
def save_submission(y_pred):
    submission = pd.DataFrame({'Id' : test_data['Id'],
                               'Prediction' : np.int64(y_pred)})
    submission.to_csv("submission.csv", index=False)

In [211]:
test_data['html_len'] = test_data['PageBase64'].apply(lambda x: len(x))

In [209]:
train_data[train_data['Prediction'] == 1]['html_len'].mean()

26730.145534193234

In [846]:
a = 0

In [ ]:
%%time
test_data['img_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'img'))
train_data['img_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'img'))

test_data['href_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'a'))
train_data['href_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'a'))

train_data['title'] = train_data['PageBase64'].apply(lambda html: get_tag(html, 'title'))
train_data['title_len'] = train_data['title'].apply(lambda x: len(x))

train_data[train_data['Prediction'] == 0]['title_len'].mean()
train_data[train_data['Prediction'] == 0]['href_number'].median()

In [100]:
%%time 
#main upload
train_data['br_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'br'))
test_data['br_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'br'))

train_data['href_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'a'))
test_data['href_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'a'))

train_data['img_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'img'))
test_data['img_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'img'))

train_data['span_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'span'))
test_data['span_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'span'))

train_data['table_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'table'))
test_data['table_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'table'))

train_data['html_len'] = train_data['PageBase64'].apply(lambda x: len(x))
test_data['html_len'] = test_data['PageBase64'].apply(lambda x: len(x))

In [960]:
train_data.to_csv("train_data_features.csv", index=False)
test_data.to_csv("test_data_features.csv", index=False)

In [199]:
%%time
train_data['a_number'] = train_data['PageBase64'].apply(lambda html: count_tag(html, 'a'))

37500
38000
38500
39000
39500
40000
40500


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


41000
41500
42000
42500
43000
43500
44000
CPU times: user 3min 5s, sys: 124 ms, total: 3min 5s
Wall time: 3min 5s


In [845]:
train_data[train_data['Prediction'] == 1]['tr_number'].mean()

18.56607447067413

In [2]:
%%time
test_data['tr_number'] = test_data['PageBase64'].apply(lambda html: count_tag(html, 'tr'))

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 10.3 µs


In [221]:
a = 0

In [19]:
def html2text_bs(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста
    """
    global a
    a += 1
    if (a % 500 == 0):
        print(a)
    soup = BeautifulSoup(raw_html, "html.parser")
    [s.extract() for s in soup(['script', 'style'])]
    return soup.get_text()

def html2text_bs_visible(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста, который видим пользователю
    """
    global a
    a += 1
    if (a % 500 == 0):
        print(a)
    soup = BeautifulSoup(raw_html, "html.parser")    
    [s.extract() for s in soup(['[document]', 'head', 'title', 'h1', 'h2', 'h3', 'body'])]
    return soup.get_text()

In [34]:
def get_tag(html, tag):
    
    soup = BeautifulSoup(html, 'html')
    text = str()
    
    for searched_tag in soup.find_all(tag):
        text += searched_tag.text + ' '
        
    return text.lower()

In [ ]:
def get_tag_stats(html, tag):
    
    soup = BeautifulSoup(html, 'html')
    text = str()
    
    for searched_tag in soup.find_all(tag):
        text += searched_tag.text + ' '
        
    return text.lower()

In [27]:
a = 0

In [95]:
def bad_words_search(html, bad_words):
 
    global a
    a += 1
    if (a % 500 == 0):
        print(a)

    count = 0
    text = str()
    
    soup = BeautifulSoup(html, 'html')
    
    for i in soup.find_all(['a', 'h1', 'title']):
        text += i.text + ' '
#     print(text)   
#     for i in soup.find_all('meta'):

#         if type(i.get('content')) == str:
# #             print(i)
#             text += i.get('content') + ' '
        
    for i in bad_words:
        count += len(re.findall(i, text.lower()))
#     print(count)   
    return count

In [75]:
bad_words = ["знакомства", "порно", "пизда", "негр", "18+", "член", "инцест", "проститутка"\
             "трахать", "трахаться", "скачать", "бесплатно", "купить", "голые", "сиськи", "грудь",\
             "порево", "порнуха", "free", "pron", "dick", "sex", "penetration",\
             "страпон", "индивидуалки", "оргия", "групповуха", "бдсм", "продать", "fuck", "fucking",\
             'секс', 'проститут','проститутки',\
             'минет', 'пенис',\
             'трах', 'трахать', 'трахнул',\
             'проститутка', 'шлюх', 'шлюхи', 'шлюха',\
             'вагина', 'срочн', 'порев', 'жопа',\
             'очко', 'измена', 'жоп',\
             'шок', 'срочно', 'продам','оргия',\
             'куртизанка', 'куртизанки', 'бдсм',\
             'сперма', 'дедовский',\
             'драть', 'дерут', 'жестко', 'жёстко', 'жесть', ' сучка', 'porn', 'anal',\
             'prostitute', \
             'blowjob', 'penis', \
             'husband', 'dick', 'wife', 'fucked', \
             'prostitute', 'whores', 'whore', \
             'vagina', 'cunt', 'gang', 'bang', \
             'treason', 'ass', 'download', \
             'shock', 'urgently', 'sell', 'buy', 'orgy', \
             'courtesan', 'mommy', 'courtesans', 'bdsm', \
             'sperm', 'old-fashioned',\
             'dating', 'to tear', 'tear', 'hard']

In [101]:
%%time
train_data['bad_words_num'] = train_data['PageBase64'].apply(lambda html: bad_words_search(str(html), bad_words))

48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
CPU times: user 4min 28s, sys: 4.29 s, total: 4min 32s
Wall time: 4min 32s


In [77]:
train_data['bad_words_num'].sum()

162872

In [78]:
train_data['bad_words_num'][8]

82

In [236]:
h = test_data['PageBase64'][4]

In [237]:
get_tag(h, 'a')

'\n узнать номер симки красивыемобильныеномера база данных телефонов организаций и частных лиц города алматы североуральск телефонный справочник как узнать свой номер телефона мегафон телефонный справочник задонска бесплатно североуральск телефонный справочник как узнать свой номер телефона мегафон местонахождение судна limnos телефонный справочник задонска бесплатно как узнать воровыный телефон найти человека поменявшего фамилию нахождение транспортированной матрицы найти положение телефона телефонный справочник актобе 2010 скачать как найти человека зная только его имя сколько стоит найти человека телефонные номера частные петербург телефонный справочник по жителям г москвы телефонная база даных москвы нахождение проекции точек телефонный справочник г астрахани скачать '

In [104]:
train_data['a_text'] = train_data['PageBase64'].apply(lambda x: get_tag(x, 'a'))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [116]:
test_data['a_text']

0        нажмите сюда для перехода на новый сервер карт...
1        в начало фотки личное о блоге контакты read fu...
2         главная контакты о проекте rss  адд kinder.ru...
3        стартовая в избранное карта сайта на главную о...
4        \n узнать номер симки красивыемобильныеномера ...
                               ...                        
16050          связаться с автором логические игры  игр...
16051    психолог в цвснп 'строительство коттеджа "под ...
16052     а к ц и я о салоне коллекция услуги инфо наши...
16053      spiele  games  игры  ігри казуал флеш все иг...
16054    tunimadeusio.narod.ru главная бесплатно пилоне...
Name: a_text, Length: 16055, dtype: object

In [117]:
test_data['a_text_len'] = test_data['a_text'].apply(lambda x: len(str(x)))

In [33]:
train_data['text_enters_num'] = train_data['text'].apply(lambda x: x.count('/n'))

In [950]:
train_data

,Id,Prediction,Url,PageBase64,br_number,href_number,img_number,span_number,table_number,html_len,...,visible_text_avg_word_len,visible_text_enters_num,a_text_len,a_text,title_text,title_len,bad_words_num,tr_number,encoded_len,compression_coef
0,-9222401963271173253,0,http://lawleader.ru/docs/32/,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...",167,59,8,5,1,62268,...,0.0,5,1459,главная о компании услуги бизнес-договоры ново...,"договор займа, договоры, договора",34,1,1,13622,0.218764
1,-9221399504663603656,0,http://alterhost.ruwww.alterhost.ruwww.user.kz...,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...",1,80,60,46,0,39936,...,0.0,4,1337,главная форумы и обсуждения блоги творчество о...,uzru – миры общения,21,2,0,10187,0.255083
2,-9219733223042265364,0,http://www.ferio.ru/c_1-acura-legend.html,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01...",69,138,5,1,38,44465,...,0.0,4,2219,\nсанкт-петербург \nсибирский фо \nюжный фо \n...,ferio.ru: ремонт и автозапчасти acura legend,46,4,149,8743,0.196627
3,-9218561774155397294,0,http://fshops.otzyv.ru/read.php?id=277,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 ...",68,72,7,7,57,29931,...,0.0,4,1016,форумы туризм кино авто магазины контакты фор...,как повесить штору?,20,1,108,7013,0.234306
4,-9216163564171210203,1,http://penhea895.narod.ru/familiya-shushmarche...,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...",4,79,7,33,3,16707,...,0.0,7,1246,рамблер поиск ещё авторамблер аудио афиша блог...,фамилия шушмарченко - французские фамилии,42,0,3,4725,0.282816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7039,9214475703562853998,1,http://glenadhy.tk/intimnie-znakomstva-v-smole...,b'<html>\r\n<head>\r\n<title>Lifemeet \xd1\x81...,34,47,14,1,7,20917,...,0.0,2,956,сайты знакомств в донецке знакомства г алмат...,lifemeet социальная сеть секс знакомств - реа...,76,40,15,6527,0.312043
7040,9214944781406040665,1,http://eicoepru.co.cc/probit-vin-pontiac/rossh...,"b'<html xmlns=""http://www.w3.org/1999/xhtml"">\...",2,15,4,0,3,15261,...,0.0,3,425,здесь пробить по vin производство авто vin код...,росшифровка vin daim — vin автомобилей утоплен...,52,0,10,5456,0.357513
7041,9217854286139025791,1,http://myflyuux.co.cc/96/46.html,b'\r\n\r\n\r\n\r\n<html>\r\n<head>\r\n<title>\...,5,20,3,1,1,15361,...,0.0,5,628,\n телефонный справочник киева по фамилии поис...,ялта.телефонный справочник | детективное агенс...,50,0,5,5304,0.345290
7042,9220612737580945795,1,http://nirpeaqo.20x.cc/?page=59,b'<html><head><title>\xd0\x93\xd0\xbe\xd1\x81\...,41,169,0,0,12,28990,...,0.0,0,6648,http://zzsklad.nm.ru/#639 скачать драйвер acor...,гостевуха сайта nirpeaqo.20x.cc - страница 59,46,93,50,8325,0.287168


In [181]:
def url_len(url):
    smth = re.search("://[^/]*/", url)
    if type(smth) == re.Match:
        print(smth.group())
        u_l = len(smth.group()[3:-1])
    else:
        u_l = 15
    
    return u_l

In [182]:
def good_site(url):
    if (url.count('.ru') > 0) | (url.count('.com') > 0):
        return 1
    else:
        return 0

In [183]:
url_len('http://lawleader.ru/docs/32/')

://lawleader.ru/


12

In [126]:
train_data['Url'] = train['Url']

In [1]:
train_data['url_len'] = train_data['Url'].apply(lambda url: url_len(url))

In [168]:
train_data['url_l']

0       28
1       50
2       41
3       38
4       54
        ..
7039    62
7040    67
7041    32
7042    31
7043    47
Name: url_l, Length: 7044, dtype: int64

In [956]:
test_data['title>12'] = test_data['title_text'].apply(lambda x: 1 if len(x.split())>=12 else 0)

In [146]:
def avg_word_len(x):
    words = x.split()
    lens = []
    for word in words:
        lens.append(len(word))
 
    return np.mean(lens)

In [59]:
i=8
html = train_data['PageBase64'][i]
train_data['Prediction'][i]

1

In [60]:
train_data['PageBase64'][8]
soup = BeautifulSoup(html, 'html')
# soup

In [99]:
# for i in soup.find_all('meta'):
#     print(i)
#     print('-------------------')
#     print(i.get('content'))
#     print('-------------------')

In [638]:
code = re.findall("charset.*", soup.find('meta')['content'])
# if code.lower() == 'windows-1251':
#     code = "utf-8"
code

['charset=windows-1251']

In [793]:
bad_words_search(html, bad_words)


0


0

In [536]:
for i in soup.find_all('meta'):
    if (type(i.charset) == str):
        charset = i.charset

In [10]:
# htmlmin.minify(html.decode(code), remove_empty_space=True)

In [821]:
# import htmlmin
# minified = htmlmin.minify(html, remove_empty_space=True)

In [577]:
a = 0

In [920]:
import slimmer

from slimmer import html_slimmer # or xhtml_slimmer, css_slimmer

ModuleNotFoundError: No module named 'slimmer'

In [921]:
import gzip

In [924]:
def encode(x):
    global a
    a+=1
    if a%500 ==0:
        print(a)
#     soup = BeautifulSoup(x, 'html')
    
#     code = re.findall("charset.*", soup.find('meta')['content'])[0][8:]
#     type(code)
#     if (code == 'None') | (code == 'windows-1251'):
#         code = "utf-8"
#     print(code)
    
#     try:
#         html=htmlmin.minify(html, remove_comments=True, remove_empty_space=True)
#     except:
#         html=html_slimmer( html.strip().replace('\n',' ').replace('\t',' ').replace('\r',' ')  )
#     return len(htmlmin.minify(x.decode("utf-8", errors='xmlcharrefreplace'), remove_empty_space=True))
    return len(gzip.compress(x))

In [ ]:
test_data['encoded_len'] = test_data['PageBase64'].apply(encode)

In [ ]:
test_data['compression_coef'] = test_data['encoded_len'] / test_data['html_len']

In [943]:
train_data[['Prediction', 'compression_coef']]

,Prediction,compression_coef
0,0,0.218764
1,0,0.255083
2,0,0.196627
3,0,0.234306
4,1,0.282816
...,...,...
7039,1,0.312043
7040,1,0.357513
7041,1,0.345290
7042,1,0.287168


In [923]:
len(gzip.compress(train_data['PageBase64'][0]))

13622

In [121]:
train_data[train_data['Prediction'] == 1]['a_text_len'].median()

737.0

In [ ]:
train_data['terminals_num'] = train_data['PageBase64'].apply(

In [81]:
train_data.drop([], axis='columns', inplace=True)

In [889]:
train_data['terminals_num'] = train_data['PageBase64'].apply(lambda x: x.decode("utf-8", errors='replace').count('/n') + \
                                                                       x.decode("utf-8", errors='replace').count('/t') + \
                                                                       x.decode("utf-8", errors='replace').count('/r'))

In [11]:
%%time
train_data['body'] = train_data['PageBase64'].apply(lambda html: get_tag(html, 'body'))
train_data['body_len'] = train_data['body'].apply(lambda x: len(x.split(sep=' '))

In [113]:
train_data['body_len'] = train_data['body'].apply(lambda x: len(x.split(sep=' ')))

In [154]:
train_data[train_data['Prediction'] == 1]['url_good'].median()

1.0

In [896]:
train_data.drop(['terminals_num'], axis='columns', inplace=True)

In [190]:
train_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

y_train = np.asarray(train_data['Prediction'])
X_train = np.asarray(train_data[['href_number', 'img_number', 'span_number', 'table_number', \
                                 'br_number', 'html_len', 'text_len', 'text_words_num',\
                                 'text_enters_num', 'text_avg_word_len', 'visible_text_len', \
                                 'visible_text_enters_num', 'visible_text_avg_word_len', 'a_text_len', 'title_len',\
                                 'bad_words_num', 'tr_number', 'url_good', 'url_len', 'url_l']])
                                 #, 'compression_coef']]) #b

y_test = np.asarray(test_data['Prediction'])
X_test = np.asarray(test_data[['href_number', 'img_number', 'span_number', 'table_number',\
                               'br_number', 'html_len', 'text_len', 'text_words_num',\
                               'text_enters_num', 'text_avg_word_len', 'visible_text_len', \
                               'visible_text_enters_num', 'visible_text_avg_word_len', 'a_text_len', 'title_len',\
                               'bad_words_num', 'tr_number']])

In [186]:
train_data

,href_number,img_number,span_number,table_number,br_number,html_len,text_avg_word_len,visible_text_len,visible_text_enters_num,visible_text_avg_word_len,...,text_words_num,text_enters_num,a_text,a_text_len,Prediction,bad_words_num,Url,url_len,url_good,url_l
0,59,8,5,1,167,62268,6.868949,5,5,0.0,...,2892,0,ГлавнаяО компанииУслугиБизнес-договорыНовостиЮ...,1400,0,0,http://lawleader.ru/docs/32/,12,1,28
1,80,60,46,0,1,39936,6.002571,4,4,0.0,...,389,0,ГлавнаяФорумы и обсужденияБлогиТворчествоОткры...,1257,0,1,http://alterhost.ruwww.alterhost.ruwww.user.kz...,42,1,50
2,138,5,1,38,69,44465,6.781188,4,4,0.0,...,1010,0,\nСанкт-Петербург\nСибирский ФО\nЮжный ФО\nУра...,2081,0,4,http://www.ferio.ru/c_1-acura-legend.html,12,1,41
3,72,7,7,57,68,29931,6.602020,4,4,0.0,...,495,0,ФорумыТуризмКиноАвтоМагазиныКонтактыФорум по м...,944,0,1,http://fshops.otzyv.ru/read.php?id=277,15,1,38
4,79,7,33,3,4,16707,7.735065,9,7,0.0,...,385,2,РамблерПоискещёАвторамблерАудиоАфишаБлогиГорос...,1167,1,0,http://penhea895.narod.ru/familiya-shushmarche...,18,1,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7039,47,14,1,7,34,20917,6.442169,2,2,0.0,...,830,0,сайты знакомств в донецкезнакомства г алматызн...,909,1,1,http://glenadhy.tk/intimnie-znakomstva-v-smole...,11,0,62
7040,15,4,0,3,2,15261,5.867861,3,3,0.0,...,893,0,здесьПробить по vin производство автоVin код p...,410,1,0,http://eicoepru.co.cc/probit-vin-pontiac/rossh...,14,0,67
7041,20,3,1,1,5,15361,6.794074,5,5,0.0,...,675,0,\nтелефонный справочник киева по фамилиипоиск ...,608,1,0,http://myflyuux.co.cc/96/46.html,14,0,32
7042,169,0,0,12,41,28990,6.221989,0,0,0.0,...,1428,0,http://zzsklad.nm.ru/#639Скачать драйвер acorp...,6479,1,5,http://nirpeaqo.20x.cc/?page=59,15,0,31


In [187]:
y_train

array([0, 0, 0, ..., 1, 1, 0])

In [188]:
params = {'learning_rate': [0.08],
          'n_estimators' : [2000]}
th = [0.5, 0.51, 0.52]

In [191]:
best = validation(X_train, y_train, GradientBoostingClassifier, params, KFold, thresholds=th)

{'learning_rate': 0.08, 'n_estimators': 2000}
th =  0.5
0.955809800838486
th =  0.51
0.9556754920712767
th =  0.52
0.9559640523502079
--------max-------
(0.9559640523502079, {'learning_rate': 0.08, 'n_estimators': 2000}, 0.52)


In [958]:
y_pred = predict(X_train, X_test, y_train, GradientBoostingClassifier, StandardScaler, best[2], **best[1])

Threshold =  0.52


In [959]:
save_submission(y_pred)

In [362]:
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [351]:
params = {'algorithm': ['ball_tree', 'brute', 'kd_tree'],
# params = {'algorithm': ['ball_tree'],
# params = {'algorithm': ['auto'],
#           'leaf_size': [5, 10, 20, 40],
#           'n_neighbors': [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
          'n_neighbors': [10, 15, 20],
#           'weights': [smart_weights],
          'p': [1],
          'n_jobs': [-1]}

In [365]:
# th = [0.41] 
params = {'n_estimators': [100, 200, 300, 400, 800], 
#           'criterion': ['gini', 'entropy'], 
          'criterion': ['gini'], 
          'max_depth': [9], 
#           'max_depth': [10, 15, None], 
          'n_jobs': [-1]}


In [361]:
# params = {'loss': ['log'],
#           'alpha': [0.01, 0.005]}
th = [0.26, 0.27, 0.28, 0.29, 0.3, 0.29, 0.3,0.31,0.32,0.33, 0.34, 0.35, 0.36, 0.37, 0.38]# 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45]

In [366]:
best = validation(X_train, y_train, RandomForestClassifier, params, KFold, thresholds=th)

{'n_estimators': 100, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}
th =  0.5
0.9295379954556189
th =  0.51
0.9302650805547319
th =  0.52
0.9305821286745275
{'n_estimators': 200, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}
th =  0.5
0.9291400707185211
th =  0.51
0.931716430624034
th =  0.52
0.93259358293496
{'n_estimators': 300, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}
th =  0.5
0.9299713873929379
th =  0.51
0.9309767644480216
th =  0.52
0.9310159562207924
{'n_estimators': 400, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}
th =  0.5
0.930250886566002
th =  0.51


KeyboardInterrupt: 

In [343]:
y_pred = predict(X_train, X_test, y_train, GradientBoostingClassifier, StandardScaler, best[2], **best[1])

Threshold =  0.52


In [474]:
import torch
import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.determenistic = True

In [489]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [490]:
X_val = X_train[7000:]
X_train = X_train[:7000]
y_val = y_train[7000:]
y_train = y_train[:7000]

In [491]:
class nnw(torch.nn.Module):
    def __init__(self, params_shape, n_hidden_neurons):
        super(nnw, self).__init__()
        self.fc1 = torch.nn.Linear(params_shape, 4*n_hidden_neurons)
        self.activ1 = torch.nn.Tanh()
        
        self.fc2 = torch.nn.Linear(4*n_hidden_neurons, 3*n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        
        self.fc3 = torch.nn.Linear(3*n_hidden_neurons, n_hidden_neurons)
        self.activ3 = torch.nn.Tanh()
        
        self.fc4 = torch.nn.Linear(n_hidden_neurons, 2)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        x = self.activ3(x)
        x = self.fc4(x)
        
        return x
    
    def final(self, x):
        x = self.forward(x)
        x = self.sm(x)
        
        return x

In [492]:
nn = nnw(X_train.shape[1], 10)

In [497]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(nn.parameters(), lr=5.0e-4)

In [498]:
np.random.permutation(5)

array([2, 1, 3, 0, 4])

In [499]:
y_train = y_train.long()

In [500]:
batch_size = X_train.shape[0]

for epoch in range(100000):
    order = np.random.permutation(len(X_train))
    
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index : start_index + batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
#         print(x_batch, y_batch)
        preds = nn.forward(x_batch)
#         print(preds, y_batch)
        loss_val = loss(preds, y_batch)
        loss_val.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = nn.forward(X_val)
        test_preds = test_preds.argmax(dim=1)
        print(metrics.f1_score(y_val, test_preds, average='weighted'))

0.861038961038961
0.861038961038961
0.861038961038961
0.861038961038961
0.8359827786057296
0.861038961038961
0.8359827786057296
0.8359827786057296
0.8100609068351005
0.8636363636363636
0.8636363636363636
0.8359827786057296
0.8359827786057296
0.8395143722437702
0.8359827786057296
0.8395143722437702
0.8359827786057296
0.8100609068351005
0.8359827786057296
0.8100609068351005
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296
0.8359827786057296


KeyboardInterrupt: 

In [484]:
test_preds = nn.forward(X_test)
y_pred = test_preds.argmax(dim=1)

In [123]:
train_data[['a_text_len']].to_csv('a_len_train_for_misha.csv', index=False)

In [102]:
train_data[['a_text']]

,a_text
0,\xd0\x93\xd0\xbb\xd0\xb0\xd0\xb2\xd0\xbd\xd0\x...
1,\xd0\x93\xd0\xbb\xd0\xb0\xd0\xb2\xd0\xbd\xd0\x...
2,\n\xd0\xa1\xd0\xb0\xd0\xbd\xd0\xba\xd1\x82-\xd...
3,\xd0\xa4\xd0\xbe\xd1\x80\xd1\x83\xd0\xbc\xd1\...
4,\xd0\xa0\xd0\xb0\xd0\xbc\xd0\xb1\xd0\xbb\xd0\x...
...,...
7039,\xd1\x81\xd0\xb0\xd0\xb9\xd1\x82\xd1\x8b \xd...
7040,\xd0\xb7\xd0\xb4\xd0\xb5\xd1\x81\xd1\x8c \xd0\...
7041,\r\n \xd1\x82\xd0\xb5\xd0\xbb\xd0\xb5\xd1\x84\...
7042,http://zzsklad.nm.ru/#639 \xd0\xa1\xd0\xba\xd0...


In [107]:
train_data

,href_number,img_number,span_number,table_number,br_number,html_len,text_avg_word_len,visible_text_len,visible_text_enters_num,visible_text_avg_word_len,...,tr_number,PageBase64,text,text_len,text_words_num,text_enters_num,a_text,a_text_len,Prediction,bad_words_num
0,59,8,5,1,167,62268,6.868949,5,5,0.0,...,1,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...","\n\n\n\nДоговор займа, договоры, договора\n\n\...",23270,2892,0,ГлавнаяО компанииУслугиБизнес-договорыНовостиЮ...,10336,0,0
1,80,60,46,0,1,39936,6.002571,4,4,0.0,...,0,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...",\n\n\n\n UzRu – миры общения\n\n\n\n\n\n\n\n\n...,3051,389,0,ГлавнаяФорумы и обсужденияБлогиТворчествоОткры...,8064,0,1
2,138,5,1,38,69,44465,6.781188,4,4,0.0,...,149,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01...",\n\nFERIO.RU: ремонт и автозапчасти Acura Lege...,8475,1010,0,\nСанкт-Петербург\nСибирский ФО\nЮжный ФО\nУра...,7884,0,4
3,72,7,7,57,68,29931,6.602020,4,4,0.0,...,108,"b'<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 ...",\n\n\n\nКак повесить штору?\n\n\n\n\n\n\n\n\n\...,4023,495,0,ФорумыТуризмКиноАвтоМагазиныКонтактыФорум по м...,5588,0,1
4,79,7,33,3,4,16707,7.735065,9,7,0.0,...,3,"b'<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0...",Фамилия Шушмарченко - французские фамилии Р...,3433,385,2,РамблерПоискещёАвторамблерАудиоАфишаБлогиГорос...,7518,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7039,47,14,1,7,34,20917,6.442169,2,2,0.0,...,15,b'<html>\r\n<head>\r\n<title>Lifemeet \xd1\x81...,\n\nLifemeet социальная сеть секс знакомств -...,6352,830,0,сайты знакомств в донецкезнакомства г алматызн...,6374,1,1
7040,15,4,0,3,2,15261,5.867861,3,3,0.0,...,10,"b'<html xmlns=""http://www.w3.org/1999/xhtml"">\...",\n\nРосшифровка vin daim — Vin автомобилей уто...,6237,893,0,здесьПробить по vin производство автоVin код p...,2413,1,0
7041,20,3,1,1,5,15361,6.794074,5,5,0.0,...,5,b'\r\n\r\n\r\n\r\n<html>\r\n<head>\r\n<title>\...,\n\n\nЯлта.телефонный справочник | детективное...,5409,675,0,\nтелефонный справочник киева по фамилиипоиск ...,4369,1,0
7042,169,0,0,12,41,28990,6.221989,0,0,0.0,...,50,b'<html><head><title>\xd0\x93\xd0\xbe\xd1\x81\...,Гостевуха сайта nirpeaqo.20x.cc - Страница 59В...,10332,1428,0,http://zzsklad.nm.ru/#639Скачать драйвер acorp...,31246,1,5
